# Описание проекта

**Заказчик:** Интернет-магазин «Викишоп».

**Цель:** Создание инструмента для осуществления поиска токсичных комментариев для последующего направления их на модерацию.

**Подцель:** создать модель машинного обучения (далее - МО) и обучить МО классифицировать комментарии на позитивные и негативные. Значение метрики качества F1 не должно быть ниже 0.75.

**Данные:** набор данных с разметкой о токсичности правок.

# 0 Подготовка ноутбука к работе

Imports

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier

from sklearn.pipeline import Pipeline

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kseniagabrusevich/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kseniagabrusevich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def display_main_info(df):
    print('________________Head________________')
    display(df.head(15))
    print('________________Data Types Info________________')
    display(df.info())
    print('________________Shape________________')
    display(df.shape)
    print('________________Duplicates________________')
    display(df.duplicated().sum())
    print('________________Proportion of Missing Data________________')
    display(df.isna().mean())

# 1 Загрузка и анализ данных

In [4]:
df= pd.read_csv('https://toxic_comments.csv', index_col=0)

In [5]:
display_main_info(df)

________________Head________________


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


________________Data Types Info________________
<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


None

________________Shape________________


(159292, 2)

________________Duplicates________________


0

________________Proportion of Missing Data________________


text     0.0
toxic    0.0
dtype: float64

In [6]:
df['toxic'].value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

**Наблюдения:**

- Число наблюдений 159292.
- Пропусков и дубликатов (полных) не обнаружено.
- Данные размечены: таргет `toxic`: 1 - комментарий токсичный, 0 - комментарий не токсичный. При этом, есть большой дисбаланс классов, токсичные комментарии составляют только 10% датасета. Это необходимо учесть при формировании выборок и в гиперпараметрах модели.

# 2 Подготовка данных

Подготовка данных будет включать в себя очистку текстов от специальных символов, токенизацию и лемматизацию.

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [9]:
df['text_preprocessed'] = df['text'].apply(preprocess_text)

In [10]:
def tokenize_lemmatize(text):

  # Tokenization
  tokenizer = WhitespaceTokenizer()
  tokens = tokenizer.tokenize(text)

  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]

  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  lemmed_tokens = [lemmatizer.lemmatize(word) for word in tokens]

  return ' '.join(lemmed_tokens)

In [11]:
df['lemmed_text'] = df['text_preprocessed'].apply(tokenize_lemmatize)

In [12]:
df.sample(5)

,text,toxic,text_preprocessed,lemmed_text
128856,Here's why it has to be analytic not just at ...,0,heres why it has to be analytic not just at i...,here analytic imagine analytic within informat...
44162,(unblock/ why is my computer blocked i coe hom...,0,unblock why is my computer blocked i coe home ...,unblock computer blocked coe home hard day wor...
113775,Further please note that the vitiated nature o...,0,further please note that the vitiated nature o...,please note vitiated nature called documentary...
15070,"Hi\n\nLook, I am still here, and always will b...",0,hi look i am still here and always will be en...,hi look still always enter fruitful dialogue l...
48784,Expanded a bit. Will see if I can find out a b...,0,expanded a bit will see if i can find out a bi...,expanded bit see find bit


In [13]:
df_final = df.drop(columns=['text', 'text_preprocessed'])

# 3 Создание и обучение моделей

Моделирование будет осуществляться с помощью векторизации текстов инструментом TfidfVectorizer.

В качестве моделей будут тестироваться:

- Logistic Regression Classifier - хорошая базовая модель. Быстрая, интерпретируемая, устойчива к переобучению при использовании регуляризации. Дополнительно, применим логарифмическое преобразование TF. 

- Passive Aggressive Classifier. Эффективна для больших текстовых данных и задач с дисбалансом классов.

In [14]:
features = df_final['lemmed_text']
target = df_final['toxic']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    test_size=0.2,
    stratify=target, 
    random_state=42
)

In [17]:
def train_model(classifier, vectorizer, hyperparametres, X, y):

  if len(X) == 0 or len(y) == 0:
      raise ValueError("X и y не должны быть пустыми.")

  model_pipe = Pipeline(steps=[
      ('vectorizer', vectorizer),
      ('classifier', classifier)
  ])


  model_search = RandomizedSearchCV(
      estimator=model_pipe,
      param_distributions=hyperparametres,
      n_iter=15,
      scoring='f1',
      n_jobs=-1,
      cv=5,
      verbose=0,
      random_state=42
      )

  model_search.fit(X, y)

  print('\nЛучшая метрика на кросс-валидации: ', model_search.best_score_)
  print('\nЛучшая конфигурация', model_search.best_params_)

  result = model_search.best_estimator_

  return result


In [ ]:
# Vecorizer 

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    sublinear_tf=True
)

In [18]:
# Logistic Regression

lr = LogisticRegression(
    random_state=42,
    solver='liblinear',
    max_iter=1000,
    class_weight='balanced'
)

params_lr = {
        'vectorizer__max_features': [10000, 50000],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__C': [0.1, 1.0, 10]
}

log_reg = train_model(
    classifier=lr,
    vectorizer=vectorizer,
    hyperparametres=params_lr,
    X=X_train,
    y=y_train
    )

/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 12 is smaller than n_iter=15. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Лучшая метрика на кросс-валидации:  0.7624446832776727

Лучшая конфигурация {'vectorizer__max_features': 50000, 'classifier__penalty': 'l2', 'classifier__C': 10}


In [19]:
# Passive Aggressive Classifier

pac = PassiveAggressiveClassifier(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'
)

params_pac = {
    'vectorizer__max_features': [10000, 50000],
    'classifier__C': [0.01, 0.1, 1.0, 10],  
    'classifier__early_stopping': [True, False],  
    'classifier__average': [True, False]  
}

pac_model = train_model(
    classifier=pac,
    vectorizer=vectorizer,
    hyperparametres=params_pac,
    X=X_train,
    y=y_train
)


Лучшая метрика на кросс-валидации:  0.7598825233502954

Лучшая конфигурация {'vectorizer__max_features': 50000, 'classifier__early_stopping': False, 'classifier__average': True, 'classifier__C': 0.1}


Логистическая регрессия показала себя лучше. Проверим качество на тестовой выборке.

In [20]:
y_preds = log_reg.predict(X_test)
f_1 = f1_score(y_test, y_preds)
print('Скор на тесте:', f_1)

Скор на тесте: 0.765496315561335


Удалось хорошо обучить модель при этом получив нужную метрику на тестовой выборке и избежать переобучения.

**Выводы по проекту:**

- В рамках данного проекта был очищен и подготовлен датасет с набором твитов.
- Проведена векторизация текстов и выбрана оптимальная модель в следующей кофигурации:

    - `vectorizer__max_features`: 50000, 
    - `classifier__penalty`: 'l2', 
    - `classifier__C`: 10
- Метрика на тествой выборке составила ~**0.77**